## Лаба 4. Прогнозирование пола и возрастной категории — Spark Streaming

#### Дедлайн

Понедельник, 22 марта, 23:59.

#### Дедлайн Github

Четверг, 25 марта, 23:59

#### Задача

У вас имеются данные: логи посещения пользователей по разным сайтам рунета. По этим пользователям вам также известны: пол и возрастная категория. Вам нужно будет в real-time спрогнозировать эти характеристики по пользователям, о которых у вас нет этой информации, но будут все те же самые логи посещения.

##### При решении задачи запрещено использовать библиотеки pandas, sklearn (кроме sklearn.metrics), xgboost и прочие.

#### Описание данных

Обучающая выборка, с которой вы будете работать, выглядит следующим образом:

```json
gender	age	uid	user_json
F	18-24	d50192e5-c44e-4ae8-ae7a-7cfe67c8b777	{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}
```

Она расположена на hdfs: `/labs/slaba04/`.

Поле `gender` принимает значения `F` (женщина) и `M` (мужчина).

Поле `age` принимает значения диапазона возраста: `18-24`, `25-34`, `35-44`, `45-54`, `>=55`

Поле `uid` принимает значения уникального ID пользователя (cookies): `d50192e5-c44e-4ae8-ae7a-7cfe67c8b777`.

Поле `user_json` имеет внутри json со следующей схемой данных: `{"visits": [{"url": "url1", "timestamp": "timestamp1"}, {"url": "url2", "timestamp": "timestamp2"}]}`. В нем содержатся непосредственно логи посещения пользователем страниц вместе с временной меткой посещения.

Данные, по которым вам надо будет построить прогноз (тестовая выборка), хранятся в Kafka в таком виде:

```json
{"uid": "bdd48781-8243-493d-8ae6-794050a4417f",
 "visits": "[{"url": "http://vk.com/feed", "timestamp": 1425077901001}, {"url": "http://big-cards.a5ltd.com/app/html/vk.html?api_url=http://api.vk.com/api.php&api_id=1804162&api_settings=663823&viewer_id=40849488&viewer_type=2&sid=1b77e9ca975573fed6e31746ca58ea2509f3588d918e7dab2a291f7f7579ae7dab6c21058c837b74b1fbb&secret=37339dfcbf&access_token=06c53bd8f9f9a0afad23b4e166e448495396259fc5a1c5e5243474df82591bd5d0afe877d993ea4a82a81&user_id=40849488&group_id=0&is_app_user=1&auth_key=d7c91ad4bb27e20cb3ed81810f5649db&language=0&parent_language=0&ad_info=elsdcqvcsvftaqxtawjsxht b0q8htjxuvbbjrvbnwojfji2ha8h&is_secure=0&ads_app_id=1804162_903aa103fa48b1e378&referrer=menu&lc_name=2c073eed&hash=", "timestamp": 1425077901000}]"}
```

То есть все то же самое, только без поля `gender` и `age` и немного с другой схемой данных. Название топика, в который мы будем присылать данные: `input_ivan.ivanov`, где вместо `ivan.ivanov` ваш логин от личного кабинета. Данные вам будут поступать по нажатию кнопки "Проверить" в личном кабинете. Вам топик создавать не нужно. Он будет автоматом создаваться при записи. Чистить его тоже не надо, потому что Spark Streaming хранит оффсеты, то есть знает, что он уже считывал, что еще не считывал. 

Порт брокера Kafka: `6667`. Hostname: `spark-master-1.newprolab.com` или `spark-node-1.newprolab.com`.

#### Результат

Вам в свою очередь нужно будет, считывая данные из Kafka, делать прогноз по ним в Spark Streaming и отправлять в real-time эти прогнозные значения в свой другой топик в Kafka в формате:

```json
{"uid": "fe1dba8f-3131-439f-9031-851c0da0f126", "gender": "M", "age": "25-34"}
{"uid": "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777", "gender": "F", "age": "18-24"}
```

Название топика — ваш логин в личном кабинете: `ivan.ivanov`. После отправки данных чекер будет ждать 90 секунд, чтобы вы обработали входные данные из топика `input_ivan.ivanov` и записали результаты прогноза уже в этот топик. 

#### Подсказки

Чтение из Kafka:

```python
read_kafka_params = {
    "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
    "subscribe": "input_ivan.ivanov",
    "startingOffsets": "latest"
}
kafka_sdf = spark.readStream.format("kafka").options(**read_kafka_params).load()
```

Запись в Kafka:

```python
write_kafka_params = {
   "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
   "topic": "ivan.ivanov"
}
batch_df.writeStream.format("kafka").options(**write_kafka_params)\
    .option("checkpointLocation", "streaming/chk/chk_kafka")\
    .outputMode("append").start()
```
  
Бывают проблемы при чтении старых данных, которых уже нет в кафке:
  
```
.option("failOnDataLoss", 'False')
```
  
#### Проверка
Проверка лабы устроена следующим образом:
1. Вы готовите свою модель
2. Настраиваете чтение данных из одного топика и запись в другой топик.
4. Запускаете свой скрипт в стриминговым режиме
3. Идет подключение к топику на чтение данных.
4. На странице лабы нажимаете кнопку "Проверить".
5. Вам в топик текут данные.
6. Модель обрабатывает входные данные.
7. Ваш скрипт сохраняет данные в другой топик.
8. Чекер на странице лабы выдает результат проверки.


Проверка осуществляется автоматическим скриптом на странице лабы в личном кабинете. Наш чекер в конечном итоге подключится к вашему финальному топику в Kafka и сравнит ваш прогноз с эталонным ответом. На выходе вы получите свой score. В качестве него метрика `accuracy`: доля пользователей от общего числа, по которым вы верно угадали и пол, и возрастную категорию. Если `accuracy` будет больше **0.25**, то лаба будет засчитана.

##### Особенности проверки
1. Проверка (генерация событий, обработка вашим скриптом, чтение и проверка результатов) занимают до 90 секунд, просьба ожидать (можно нажимать кнопку "обновить")
2. За один раз (одно нажание "Проверить") чекер присылает батч в 5000 событий. Если для отладки / повторной проверки требуется еще, надо нажимать снова

####  Рекомендации по прохождению
1. Разработанную модель (и вообще весь пайплайн) можно сохранить в HDFS и потом загрузить уже в стриминговом скрипте (методы save/load - см.документацию)
2. Настоятельно рекомендуется сначала отладить весь процесс чтения данных, предсказания и сохранения результатов в **батчевом** режиме (сгенерить порцию событий чекером и потом считать ее в обычный DataFrame через  `spark.read.format("kafka")`. И уже только после проверки полной  работоспособсности всего скрипта и корректности формата выходных данных в Кафке, переписать на стриминговый режим (благодаря единому API Spark, это не потребует больших изменений)

Обязательное условие зачета лабораторной работы – это выкладка после дедлайна лабы своего решения в репозиторий через pull-request. Как это сделать, можно прочитать [здесь](/git.md). Если будут вопросы – спрашивайте в Slack.

# Решение

In [209]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 10 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [210]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "Efimov Ilya lab4") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
import pickle
import pandas as pd
import numpy as np
import pyspark.sql.functions as f
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import StructType, StructField, IntegerType, StringType 
from pyspark.sql.types import TimestampType, LongType, FloatType, ArrayType
from pyspark.ml import Estimator
from pyspark.ml.linalg import VectorUDT

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

from sklearn.linear_model import LogisticRegression
from pyspark import keyword_only
from pyspark.ml import Model, Estimator
from pyspark.ml.param import Param, Params, TypeConverters
from pyspark.ml.param.shared import HasFeaturesCol, HasLabelCol, HasPredictionCol

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier, LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.feature import IDFModel

In [5]:
from urllib.parse import urlparse
import json

## train

In [5]:
!hdfs dfs -ls /labs/slaba04/

Found 1 items
-rw-r--r--   3 hdfs hdfs  655090069 2021-02-27 22:13 /labs/slaba04/gender_age_dataset.txt


In [36]:
spark.sparkContext.textFile("/labs/slaba04/gender_age_dataset.txt").take(2)

['gender\tage\tuid\tuser_json',
 'F\t18-24\td50192e5-c44e-4ae8-ae7a-7cfe67c8b777\t{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}']

In [389]:
@f.pandas_udf(IntegerType())
def encode_gender(domains):
    mapping = {
        '-': 0,
        'F': 1,
        'M': 2,
    }
    return domains.apply(lambda x: mapping.get(x))
@f.pandas_udf(StringType())
def dencode_gender(domains):
    mapping = {
        0:'-',
        1:'F',
        2:'M',
    }
    return domains.apply(lambda x: mapping.get(x))
@f.pandas_udf(IntegerType())
def encode_age(domains):
    mapping = {
        '-': 0,
        '45-54': 1,
        '35-44': 2,
        '25-34': 3,
        '18-24': 4,
        '>=55': 5,
    }
    return domains.apply(lambda x: mapping.get(x))
@f.pandas_udf(StringType())
def dencode_age(domains):
    mapping = {
        0:'-',
        1:'45-54',
        2:'35-44',
        3:'25-34',
        4:'18-24',
        5:'>=55',
    }
    return domains.apply(lambda x: mapping.get(x))

In [390]:
schema = StructType(fields=[
    StructField("gender", StringType()), # принимает значения F (женщина) и M (мужчина)
    StructField("age", StringType()), # принимает значения диапазона возраста: 18-24, 25-34, 35-44, 45-54, >=55
    StructField("uid", StringType()), #uid принимает значения уникального ID пользователя (cookies)
    StructField("user_json", StringType()) #содержатся непосредственно логи посещения пользователем страниц. 
])
df = spark.read.csv("/labs/slaba04/gender_age_dataset.txt", header=True, sep='\t', schema=schema)
df = df.withColumn("gender", encode_gender("gender"))
df = df.withColumn("age", encode_age("age")).cache()
df.show(2)

+------+---+--------------------+--------------------+
|gender|age|                 uid|           user_json|
+------+---+--------------------+--------------------+
|     1|  4|d50192e5-c44e-4ae...|{"visits": [{"url...|
|     2|  3|d502331d-621e-472...|{"visits": [{"url...|
+------+---+--------------------+--------------------+
only showing top 2 rows



In [391]:
df.printSchema()

root
 |-- gender: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)



In [392]:
df_test = df.select("*").limit(10).toPandas()

In [393]:
def parse_timestamp_visits_(x):
    x=json.loads(x)
    return np.ediff1d([i['timestamp'] for i in x['visits']]).mean()
def parse_timestamp_visits_series(x):
    return x.apply(parse_timestamp_visits_)
def parse_url_visits_(x):
    x=json.loads(x) 
    return " ".join([urlparse(i['url'].replace("http://http://","http://")
                               .replace("http://https://","http://")
                               .replace("http://http://","https://"))
                      .netloc.replace("www.","") for i in x['visits']])
def parse_url_visits_series(x):
    return x.apply(parse_url_visits_)
parse_timestamp_visits = f.pandas_udf(parse_timestamp_visits_series,"double")
parse_url_visits = f.pandas_udf(parse_url_visits_series,"string")
# df_test["user_json"].apply(parse_timestamp_visits_).values

In [394]:
df = df.withColumn("meantime", parse_timestamp_visits("user_json")).cache() 

In [395]:
df = df.withColumn("urlstring", parse_url_visits("user_json")).cache() 

In [396]:
tokenizer = Tokenizer(inputCol="urlstring", outputCol="words")
wordsData = tokenizer.transform(df)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2000)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
df = idfModel.transform(featurizedData).cache() 

df.select("urlstring", "features").show()

+--------------------+--------------------+
|           urlstring|            features|
+--------------------+--------------------+
|zebra-zoya.ru new...|(2000,[943,1215,1...|
|sweetrading.ru sw...|(2000,[38,52,63,7...|
|ru.oriflame.com r...|(2000,[231,326,70...|
|translate-tattoo....|(2000,[600,622,73...|
|mail.rambler.ru n...|(2000,[105,184,22...|
|cfire.mail.ru pet...|(2000,[392,444],[...|
|msn.com msn.com m...|(2000,[402,425,82...|
|gazprom.ru re-sto...|(2000,[172,609,70...|
|lifenews.ru lifen...|(2000,[767,1252,1...|
|google.ru films.i...|(2000,[163,568,58...|
|muz4in.net smachn...|(2000,[22,277,307...|
|kosmetista.ru kos...|(2000,[491,1157],...|
|android.mobile-re...|(2000,[176,452,59...|
|tsn.ua cfts.org.u...|(2000,[205,428,63...|
|jobinmoscow.ru jo...|(2000,[54,498,872...|
|      abc-people.com|(2000,[588],[4.80...|
|easygames.biz eas...|(2000,[602],[9.31...|
|ratanews.ru ratan...|(2000,[111,228,30...|
|        sam-zdrav.ru|(2000,[1482],[5.8...|
|     msn.com msn.com|(2000,[141

In [397]:
tokenizer.write().overwrite().save("tokenizer_model")
hashingTF.write().overwrite().save("hashingTF_model")
idfModel.write().overwrite().save("idfModel_model")

In [424]:
from pyspark.ml.classification import MultilayerPerceptronClassifier, RandomForestClassifier, RandomForestClassificationModel

In [420]:
predictions = df.limit(20) 
# lr = NaiveBayes(labelCol="gender", featuresCol="features", smoothing=1.0, modelType="multinomial")
# lr = GBTClassifier(labelCol="gender", featuresCol="features", maxIter=50, maxDepth=3) # 0.9003
lr = RandomForestClassifier(labelCol="gender", featuresCol="features") 
model = lr.fit(df.where('gender !=0'))
model.write().overwrite().save("gender_model")
predictions = model.transform(predictions)
predictions = predictions.withColumn("gender2", dencode_gender("prediction"))

columns_to_drop = ['rawPrediction', 'probability', 'prediction']
predictions = predictions.drop(*columns_to_drop)

# lr = NaiveBayes(labelCol="age", featuresCol="features", smoothing=1.0, modelType="multinomial")
# lr = GBTClassifier(labelCol="age", featuresCol="features", maxIter=50, maxDepth=3) # 0.9003
lr = RandomForestClassifier(labelCol="age", featuresCol="features") 
model = lr.fit(df.where('age !=0'))
model.write().overwrite().save("age_model")
predictions = model.transform(predictions)
predictions = predictions.withColumn("age2", dencode_age("prediction"))

In [421]:
predictions.toPandas()[["age", "age2", "gender", "gender2"]]

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


,age,age2,gender,gender2
0,2,25-34,2,M
1,3,25-34,2,M
2,3,25-34,2,M
3,3,25-34,2,M
4,3,25-34,2,M
5,3,25-34,2,M
6,3,25-34,1,F
7,3,25-34,2,M
8,3,25-34,1,M
9,4,25-34,1,M


## Kafka

In [462]:
tokenizer = Tokenizer.load("tokenizer_model")
hashingTF = HashingTF.load("hashingTF_model")
idfModel = IDFModel.load("idfModel_model")
age_model = RandomForestClassificationModel.load("age_model")
gender_model = RandomForestClassificationModel.load("gender_model")

In [489]:
KAFKA_BOOTSTRAP_SERVER = 'spark-node-1.newprolab.com:6667'
# KAFKA_BOOTSTRAP_SERVER = 'spark-master-1.newprolab.com:6667'
INPUT_KAFKA_TOPIC = 'input_ilya.efimov'
OUTPUT_KAFKA_TOPIC = 'ilya.efimov'
read_kafka_params = {
    'kafka.bootstrap.servers': KAFKA_BOOTSTRAP_SERVER,
    'subscribe': INPUT_KAFKA_TOPIC,
#     'startingOffsets': 'earliest', # убрать в стриме
    "startingOffsets": "latest", #Добавить в стриме
}
# kafka_sdf = (
#     spark
#     .read
#     .format('kafka')
#     .options(**read_kafka_params)
#     .load()
#     .cache()
# )
kafka_sdf = spark.readStream.format("kafka").options(**read_kafka_params).load()

kafka_sdf.printSchema()
# print('count',kafka_sdf.count())

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [490]:
kafka_sdf = kafka_sdf.selectExpr("CAST(value AS STRING)")#.cache()

In [491]:
def parse_url_visits_(x):
    
    x = json.loads(x)
    x = json.loads(x['visits'])
#     return(str(x))
#     try:
    return " ".join([urlparse(i['url'].replace("http://http://","http://")
                               .replace("http://https://","http://")
                               .replace("http://http://","https://"))
                      .netloc.replace("www.","") for i in x])
def parse_url_visits_series(x):
    return x.apply(parse_url_visits_) 
parse_url_visits = f.pandas_udf(parse_url_visits_series,"string") 

kafka_sdf = kafka_sdf.withColumn("urlstring", parse_url_visits("value"))#.cache()

In [492]:
wordsData = tokenizer.transform(kafka_sdf)
featurizedData = hashingTF.transform(wordsData)
kafka_sdf = idfModel.transform(featurizedData) 

kafka_sdf = gender_model.transform(kafka_sdf)
kafka_sdf = kafka_sdf.withColumn("gender", dencode_gender("prediction"))

columns_to_drop = ['rawPrediction', 'probability', 'prediction']
kafka_sdf = kafka_sdf.drop(*columns_to_drop)

kafka_sdf = age_model.transform(kafka_sdf)
kafka_sdf = kafka_sdf.withColumn("age", dencode_age("prediction"))

columns_to_drop = ['rawPrediction', 'probability', 'prediction']
kafka_sdf = kafka_sdf.drop(*columns_to_drop)#.cache() 
# kafka_sdf.show(15)

In [493]:
def parse_uid_(x):
    x = json.loads(x)
    return x['uid']

def parse_uid_series(x):
    return x.apply(parse_uid_) 
parse_uid = f.pandas_udf(parse_uid_series,"string") 

kafka_sdf = kafka_sdf.withColumn("uid", parse_uid("value")) 

In [494]:
columns_to_drop = ['value', 'urlstring', 'words', 'rawFeatures', 'features']
kafka_sdf = kafka_sdf.drop(*columns_to_drop) 

### Запись в Kafka

In [495]:
write_kafka_params = {
   "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
   "topic": "ilya.efimov"
}

doc = to_json(struct(*[col(c) for c in kafka_sdf.columns]))

sq = kafka_sdf.select(doc.alias("value")).writeStream.format("kafka").options(**write_kafka_params)\
    .option('checkpointLocation', 'streaming/chk/chk_kafka_ilya_efimov_lab04')\
    .outputMode("append").start()
sq.status

{'message': 'Initializing sources',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [502]:
sq.status

{'message': 'Waiting for next trigger',
 'isDataAvailable': False,
 'isTriggerActive': True}

In [457]:
from pyspark.sql.functions import struct, to_json
# для обычного режима
def write_kafka(topic, data):
    kafka_params = {
   "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
   "topic": "ilya.efimov"
}
    kafka_doc = to_json(struct(*[col(c) for c in data.columns]))
    data.select(kafka_doc.alias("value")) \
        .withColumn("topic", f.lit(kafka_params["topic"])) \
        .write.format("kafka") \
        .options(**kafka_params)\
        .save()
# write_kafka("test_topic0", kafka_sdf)

{'message': 'Initializing sources', 'isDataAvailable': False, 'isTriggerActive': False}


In [487]:
def kill_all():
    streams = SparkSession.builder.getOrCreate().streams.active
    for s in streams:
        desc = s.lastProgress["sources"][0]["description"]
        s.stop()
        print("Stopped {s}".format(s=desc))
kill_all()

Stopped KafkaV2[Subscribe[input_ilya.efimov]]


In [ ]:
# 0.2608

In [367]:
# 2000 - 0.2426, all - 0.197, 20000 - 0.1832, 1000 - 0.2454, 700 - 0.2444

### Проверим наличие данных в целевом топике

In [488]:
KAFKA_BOOTSTRAP_SERVER = 'spark-node-1.newprolab.com:6667'
# KAFKA_BOOTSTRAP_SERVER = 'spark-master-1.newprolab.com:6667'
INPUT_KAFKA_TOPIC = 'input_ilya.efimov'
OUTPUT_KAFKA_TOPIC = 'ilya.efimov'
read_kafka_params = {
    'kafka.bootstrap.servers': KAFKA_BOOTSTRAP_SERVER,
    'subscribe': OUTPUT_KAFKA_TOPIC,
#     'startingOffsets': 'earliest', # убрать в стриме
#     "startingOffsets": "latest", #Заменить в стриме
}
sdf2 = (
    spark
    .read
    .format('kafka')
    .options(**read_kafka_params)
    .load()
    .cache()
) 

sdf2.printSchema()
print('count',sdf2.count())
sdf2.selectExpr("CAST(value AS STRING)").show(5)

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

count 0
+-----+
|value|
+-----+
+-----+



In [ ]:
40868

In [59]:
spark.stop()